In [1]:
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tnrange, tqdm
import seaborn as sns
from joblib import dump, load
import pandas as pd
import argparse

In [2]:
with open('state_features.txt') as f:
    state_features = f.read().split()
print(state_features)
print(len(state_features))

['Albumin', 'Arterial_BE', 'Arterial_lactate', 'Arterial_pH', 'BUN', 'CO2_mEqL', 'Calcium', 'Chloride', 'Creatinine', 'DiaBP', 'FiO2_1', 'GCS', 'Glucose', 'HCO3', 'HR', 'Hb', 'INR', 'Ionised_Ca', 'Magnesium', 'MeanBP', 'PT', 'PTT', 'PaO2_FiO2', 'Platelets_count', 'Potassium', 'RR', 'SGOT', 'SGPT', 'SIRS', 'SOFA', 'Shock_Index', 'Sodium', 'SpO2', 'SysBP', 'Temp_C', 'Total_bili', 'WBC_count', 'Weight_kg', 'age', 'elixhauser', 'gender', 'mechvent', 'output_4hourly', 'output_total', 'paCO2', 'paO2', 're_admission', 'bloc']
48


In [3]:
# define an action mapping - how to get an id representing the action from the (iv,vaso) tuple
action_map = {(iv, vaso): 5*iv+vaso for iv in range(5) for vaso in range(5)}

In [ ]:
## OLD action space

In [4]:
def make_trajectories(df):
    trajectories = []
    for i, g in tqdm(df.groupby('icustayid')):
        try:
            g = g.reset_index(drop=True)
            trajectory = []
            for t in range(len(g)-1):
                transition = {
                    's': g.loc[t, state_features].values, 
                    'a': action_map[
                        int(g.loc[t, 'iv_input']),
                        int(g.loc[t, 'vaso_input'])
                    ],
                    'r': g.loc[t, 'terminal_reward'],
                    's_': g.loc[t+1, state_features].values, 
                    'a_': action_map[
                        int(g.loc[t+1, 'iv_input']),
                        int(g.loc[t+1, 'vaso_input'])
                    ],
                    'done': False,
                }
                trajectory.append(transition)
            
            t = len(g)-1
            trajectory.append({
                's': g.loc[t, state_features].values, 
                'a': action_map[
                    int(g.loc[t, 'iv_input']),
                    int(g.loc[t, 'vaso_input'])
                ], 
                'r': g.loc[t, 'terminal_reward'],
                's_': None, 
                'a_': None,
                'done': True,
            })
#             print(trajectory)
            trajectories.append(trajectory)
        except:
            print(i)
            display(g)
            raise
    return trajectories

In [5]:
# VAL
traj_va = make_trajectories(pd.read_csv('../data/rl_val_set_scaled.csv'))
pkl.dump(traj_va, open('traj_OLD_va.pkl', 'wb'))

100%|██████████| 2094/2094 [00:55<00:00, 38.00it/s]


In [6]:
# TEST
traj_te = make_trajectories(pd.read_csv('../data/rl_test_set_scaled.csv'))
pkl.dump(traj_te, open('traj_OLD_te.pkl', 'wb'))

100%|██████████| 4189/4189 [01:48<00:00, 38.71it/s]


In [7]:
# TRAIN
traj_tr = make_trajectories(pd.read_csv('../data/rl_train_set_scaled.csv'))
pkl.dump(traj_tr, open('traj_OLD_tr.pkl', 'wb'))

100%|██████████| 14657/14657 [06:23<00:00, 38.24it/s]


In [ ]:
## NEW action space

In [8]:
def make_trajectories(df):
    trajectories = []
    for i, g in tqdm(df.groupby('icustayid')):
        try:
            g = g.reset_index(drop=True)
            trajectory = []
            for t in range(len(g)-1):
                transition = {
                    's': g.loc[t, state_features].values, 
                    'a': action_map[
                        int(g.loc[t, 'iv_input_NEW']),
                        int(g.loc[t, 'vaso_input_NEW'])
                    ],
                    'r': g.loc[t, 'terminal_reward'],
                    's_': g.loc[t+1, state_features].values, 
                    'a_': action_map[
                        int(g.loc[t+1, 'iv_input_NEW']),
                        int(g.loc[t+1, 'vaso_input_NEW'])
                    ],
                    'done': False,
                }
                trajectory.append(transition)
            
            t = len(g)-1
            trajectory.append({
                's': g.loc[t, state_features].values, 
                'a': action_map[
                    int(g.loc[t, 'iv_input_NEW']),
                    int(g.loc[t, 'vaso_input_NEW'])
                ], 
                'r': g.loc[t, 'terminal_reward'],
                's_': None, 
                'a_': None,
                'done': True,
            })
#             print(trajectory)
            trajectories.append(trajectory)
        except:
            print(i)
            display(g)
            raise
    return trajectories

In [9]:
# VAL
traj_va = make_trajectories(pd.read_csv('../data/rl_val_set_scaled.csv'))
pkl.dump(traj_va, open('traj_va.pkl', 'wb'))

100%|██████████| 2094/2094 [00:55<00:00, 37.88it/s]


In [10]:
# TEST
traj_te = make_trajectories(pd.read_csv('../data/rl_test_set_scaled.csv'))
pkl.dump(traj_te, open('traj_te.pkl', 'wb'))

100%|██████████| 4189/4189 [01:46<00:00, 39.26it/s]


In [11]:
# TRAIN
traj_tr = make_trajectories(pd.read_csv('../data/rl_train_set_scaled.csv'))
pkl.dump(traj_tr, open('traj_tr.pkl', 'wb'))

100%|██████████| 14657/14657 [06:34<00:00, 37.17it/s]
